In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from utils import train
torch.cuda.empty_cache()

In [2]:
# Model definition
class CIFAR10_MLP(nn.Module):
    def __init__(self, input_dim=3*32*32, hidden_dims=[2048, 1024, 512, 256], num_classes=10, dropout=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hdim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hdim))
            layers.append(nn.BatchNorm1d(hdim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout))
            prev_dim = hdim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        return self.net(x)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transforms: normalize CIFAR-10 images
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = CIFAR10_MLP().to(device)
model = torch.jit.script(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
train_metrics, val_metrics, test_metrics = train(model, train_loader, test_loader, test_loader, 30, optimizer, criterion)


Epoch: 1 Total_Time: 1.6134 Average_Time_per_batch: 0.0041 Train_Accuracy: 0.3362 Train_Loss: 1.8267 
Epoch: 2 Total_Time: 1.2176 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.4235 Train_Loss: 1.6107 
Epoch: 3 Total_Time: 1.2475 Average_Time_per_batch: 0.0032 Train_Accuracy: 0.4552 Train_Loss: 1.5287 
Epoch: 4 Total_Time: 1.1978 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.4756 Train_Loss: 1.4754 
Epoch: 5 Total_Time: 1.2134 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.4919 Train_Loss: 1.4324 
Epoch: 6 Total_Time: 1.3463 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.5089 Train_Loss: 1.3881 
Epoch: 7 Total_Time: 1.2483 Average_Time_per_batch: 0.0032 Train_Accuracy: 0.5191 Train_Loss: 1.3609 
Epoch: 8 Total_Time: 1.2941 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5294 Train_Loss: 1.3320 
Epoch: 9 Total_Time: 1.1962 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.5396 Train_Loss: 1.3086 
Epoch: 10 Total_Time: 1.2108 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.546

In [6]:
from dpn_3.dpn import DPN as DPN_3
    
model_3 = DPN_3(3*32*32, 100, 10, True).cuda()
#model_3.compile()

In [7]:
model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, test_loader, test_loader, 30, optimizer, criterion)


Epoch: 1 Total_Time: 0.6407 Average_Time_per_batch: 0.0016 Train_Accuracy: 0.3946 Train_Loss: 1.7723 
Epoch: 2 Total_Time: 0.4856 Average_Time_per_batch: 0.0012 Train_Accuracy: 0.4566 Train_Loss: 1.5985 
Epoch: 3 Total_Time: 0.6010 Average_Time_per_batch: 0.0015 Train_Accuracy: 0.4813 Train_Loss: 1.5205 
Epoch: 4 Total_Time: 0.7240 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5030 Train_Loss: 1.4552 
Epoch: 5 Total_Time: 0.4826 Average_Time_per_batch: 0.0012 Train_Accuracy: 0.5207 Train_Loss: 1.4047 
Epoch: 6 Total_Time: 0.6000 Average_Time_per_batch: 0.0015 Train_Accuracy: 0.5407 Train_Loss: 1.3487 
Epoch: 7 Total_Time: 0.6966 Average_Time_per_batch: 0.0018 Train_Accuracy: 0.5514 Train_Loss: 1.3117 
Epoch: 8 Total_Time: 0.6234 Average_Time_per_batch: 0.0016 Train_Accuracy: 0.5625 Train_Loss: 1.2777 
Epoch: 9 Total_Time: 0.5431 Average_Time_per_batch: 0.0014 Train_Accuracy: 0.5699 Train_Loss: 1.2529 
Epoch: 10 Total_Time: 0.8166 Average_Time_per_batch: 0.0021 Train_Accuracy: 0.578

In [9]:
import torch
from torch import nn

hidden_dims = [2048, 1024, 512, 256, 10]
total = sum(hidden_dims)

blocks = len(hidden_dims)
features = 3 * 32 * 32
neural_blocks = []
for dim in hidden_dims:
    std_dev = torch.sqrt(torch.tensor(1 / features)).to(device)
    neural_blocks.append(torch.randn(dim, features).to(device) * std_dev)
    features += dim

feature_blocks = []
features_start = 0
for i in range(len(neural_blocks)):
    features_end = neural_blocks[i].shape[1]
    block = neural_blocks[i][:, features_start:]
    for j in range(i + 1, len(neural_blocks)):
        block = torch.cat((block, neural_blocks[j][:, features_start:features_end]), dim=0)
    feature_blocks.append(nn.Parameter(block))
    features_start = features_end

biases = biases = nn.Parameter(torch.empty(total).uniform_(0.0, 1.0)).to(device)

In [10]:
from dpn_2.dpn import DPN as DPN_2
    
model_2 = DPN_2(3*32*32, total, 10, True).cuda()
model_2.weights.extend(feature_blocks)
model_2.biases = biases

In [11]:
#model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_2.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
from utils import train
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_2, train_loader, test_loader, test_loader, 30, optimizer, criterion)


Epoch: 1 Total_Time: 1.4715 Average_Time_per_batch: 0.0038 Train_Accuracy: 0.3852 Train_Loss: 2.0976 
Epoch: 2 Total_Time: 1.4417 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.4435 Train_Loss: 1.7311 
Epoch: 3 Total_Time: 1.5289 Average_Time_per_batch: 0.0039 Train_Accuracy: 0.4738 Train_Loss: 1.6432 
Epoch: 4 Total_Time: 1.2728 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5034 Train_Loss: 1.5827 
Epoch: 5 Total_Time: 1.8028 Average_Time_per_batch: 0.0046 Train_Accuracy: 0.5270 Train_Loss: 1.5328 
Epoch: 6 Total_Time: 1.4883 Average_Time_per_batch: 0.0038 Train_Accuracy: 0.5470 Train_Loss: 1.4850 
Epoch: 7 Total_Time: 1.7106 Average_Time_per_batch: 0.0044 Train_Accuracy: 0.5700 Train_Loss: 1.4308 
Epoch: 8 Total_Time: 1.4225 Average_Time_per_batch: 0.0036 Train_Accuracy: 0.5829 Train_Loss: 1.4133 
Epoch: 9 Total_Time: 1.6613 Average_Time_per_batch: 0.0042 Train_Accuracy: 0.6061 Train_Loss: 1.3553 
Epoch: 10 Total_Time: 1.4293 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.618

In [13]:
class ResNetMLP(nn.Module):
    def __init__(self, num_classes=10, mlp_hidden=256):
        super().__init__()
        # Load ResNet18 (pretrained=False for CIFAR10, since ImageNet weights use 224x224)
        backbone = torchvision.models.resnet18(weights=None)
        # Change input conv layer for CIFAR-10 (3x32x32)
        backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        backbone.maxpool = nn.Identity()  # Remove the first maxpool
        # Extract up to the last layer
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # Remove FC
        self.mlp = nn.Sequential(
            nn.Linear(512, mlp_hidden),
            nn.BatchNorm1d(mlp_hidden),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(mlp_hidden, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)  # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten
        x = self.mlp(x)
        return x

In [14]:
model = ResNetMLP(num_classes=10, mlp_hidden=256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, test_loader, test_loader, 30, optimizer, criterion, False)


Epoch: 1 Total_Time: 5.6266 Average_Time_per_batch: 0.0144 Train_Accuracy: 0.5422 Train_Loss: 1.2653 
Epoch: 2 Total_Time: 5.5648 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.7298 Train_Loss: 0.7776 
Epoch: 3 Total_Time: 5.3441 Average_Time_per_batch: 0.0137 Train_Accuracy: 0.8011 Train_Loss: 0.5835 
Epoch: 4 Total_Time: 5.3052 Average_Time_per_batch: 0.0136 Train_Accuracy: 0.8426 Train_Loss: 0.4562 
Epoch: 5 Total_Time: 5.2880 Average_Time_per_batch: 0.0135 Train_Accuracy: 0.8766 Train_Loss: 0.3585 
Epoch: 6 Total_Time: 5.3063 Average_Time_per_batch: 0.0136 Train_Accuracy: 0.9096 Train_Loss: 0.2684 
Epoch: 7 Total_Time: 5.3196 Average_Time_per_batch: 0.0136 Train_Accuracy: 0.9282 Train_Loss: 0.2071 
Epoch: 8 Total_Time: 5.2947 Average_Time_per_batch: 0.0135 Train_Accuracy: 0.9491 Train_Loss: 0.1503 
Epoch: 9 Total_Time: 5.3139 Average_Time_per_batch: 0.0136 Train_Accuracy: 0.9589 Train_Loss: 0.1196 
Epoch: 10 Total_Time: 5.3056 Average_Time_per_batch: 0.0136 Train_Accuracy: 0.968

In [16]:
class ResNetDPN(nn.Module):
    def __init__(self, num_classes=10, mlp_hidden=256):
        super().__init__()
        # Load ResNet18 (pretrained=False for CIFAR10, since ImageNet weights use 224x224)
        backbone = torchvision.models.resnet18(weights=None)
        # Change input conv layer for CIFAR-10 (3x32x32)
        backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        backbone.maxpool = nn.Identity()  # Remove the first maxpool
        # Extract up to the last layer
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # Remove FC
        self.dpn = DPN_3(512, mlp_hidden + num_classes, num_classes, True).cuda()

    def forward(self, x):
        x = self.backbone(x)  # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dpn(x)
        return x

In [17]:
model = ResNetDPN(num_classes=10, mlp_hidden=256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, test_loader, test_loader, 30, optimizer, criterion, False)


Epoch: 1 Total_Time: 5.2147 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.5607 Train_Loss: 1.2100 
Epoch: 2 Total_Time: 5.2288 Average_Time_per_batch: 0.0134 Train_Accuracy: 0.7524 Train_Loss: 0.7097 
Epoch: 3 Total_Time: 5.3682 Average_Time_per_batch: 0.0137 Train_Accuracy: 0.8202 Train_Loss: 0.5154 
Epoch: 4 Total_Time: 5.2310 Average_Time_per_batch: 0.0134 Train_Accuracy: 0.8640 Train_Loss: 0.3918 
Epoch: 5 Total_Time: 5.2097 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.8987 Train_Loss: 0.2887 
Epoch: 6 Total_Time: 5.2051 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.9261 Train_Loss: 0.2071 
Epoch: 7 Total_Time: 5.2359 Average_Time_per_batch: 0.0134 Train_Accuracy: 0.9497 Train_Loss: 0.1432 
Epoch: 8 Total_Time: 5.2210 Average_Time_per_batch: 0.0134 Train_Accuracy: 0.9614 Train_Loss: 0.1097 
Epoch: 9 Total_Time: 5.5704 Average_Time_per_batch: 0.0142 Train_Accuracy: 0.9720 Train_Loss: 0.0792 
Epoch: 10 Total_Time: 5.2089 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.974

In [3]:
in_channels = 3                # change to 3 if you use CIFAR10 dataset
image_size = 32                # change to 32 if you use CIFAR10 dataset
num_classes = 10

lr = 1e-3

patch_size = 4         # Each patch is 16x16, so 2x2 = 4 patches per image
hidden_dim = 256       # Token-mixing MLP hidden dim (formerly token_dim)
tokens_mlp_dim = 512    # Tokens MLP dim
channels_mlp_dim = 2048 # Channels MLP dim
num_blocks = 6         # Number of Mixer layers

In [4]:
from MLP_Mixer import MLPMixer
model = MLPMixer(in_channels=in_channels, embedding_dim=hidden_dim, num_classes=num_classes, patch_size=patch_size, image_size=image_size, depth=num_blocks, token_intermediate_dim=tokens_mlp_dim, channel_intermediate_dim=channels_mlp_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [5]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, test_loader, test_loader, 10, optimizer, criterion, False)


Epoch: 1 Total_Time: 5.8714 Average_Time_per_batch: 0.0150 Train_Accuracy: 0.3647 Train_Loss: 1.7052 Validation_Accuracy: 0.4845 Validation_Loss: 1.3919
Epoch: 2 Total_Time: 5.2486 Average_Time_per_batch: 0.0134 Train_Accuracy: 0.5535 Train_Loss: 1.2366 Validation_Accuracy: 0.5835 Validation_Loss: 1.1568
Epoch: 3 Total_Time: 5.1849 Average_Time_per_batch: 0.0133 Train_Accuracy: 0.6309 Train_Loss: 1.0340 Validation_Accuracy: 0.6461 Validation_Loss: 0.9891
Epoch: 4 Total_Time: 5.1624 Average_Time_per_batch: 0.0132 Train_Accuracy: 0.6748 Train_Loss: 0.9124 Validation_Accuracy: 0.6582 Validation_Loss: 0.9474
Epoch: 5 Total_Time: 5.0545 Average_Time_per_batch: 0.0129 Train_Accuracy: 0.7092 Train_Loss: 0.8165 Validation_Accuracy: 0.6786 Validation_Loss: 0.8981
Epoch: 6 Total_Time: 5.0221 Average_Time_per_batch: 0.0128 Train_Accuracy: 0.7412 Train_Loss: 0.7199 Validation_Accuracy: 0.6942 Validation_Loss: 0.8681
Epoch: 7 Total_Time: 5.8173 Average_Time_per_batch: 0.0149 Train_Accuracy: 0.7753

In [6]:
from DPN_Mixer import MLPMixer as DPNMixer
model = DPNMixer(in_channels=in_channels, embedding_dim=hidden_dim, num_classes=num_classes, patch_size=patch_size, image_size=image_size, depth=num_blocks, token_intermediate_dim=tokens_mlp_dim, channel_intermediate_dim=channels_mlp_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [7]:
torch.cuda.empty_cache()

'''if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)  # Wrap the model in DataParallel'''

model = model.to(device)

In [8]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model, train_loader, test_loader, test_loader, 15, optimizer, criterion, False)


Epoch: 1 Total_Time: 3.5042 Average_Time_per_batch: 0.0090 Train_Accuracy: 0.4085 Train_Loss: 1.6121 Validation_Accuracy: 0.5114 Validation_Loss: 1.3472
Epoch: 2 Total_Time: 3.4593 Average_Time_per_batch: 0.0088 Train_Accuracy: 0.5622 Train_Loss: 1.2118 Validation_Accuracy: 0.5875 Validation_Loss: 1.1447
Epoch: 3 Total_Time: 3.4687 Average_Time_per_batch: 0.0089 Train_Accuracy: 0.6288 Train_Loss: 1.0373 Validation_Accuracy: 0.6316 Validation_Loss: 1.0269
Epoch: 4 Total_Time: 3.4828 Average_Time_per_batch: 0.0089 Train_Accuracy: 0.6691 Train_Loss: 0.9230 Validation_Accuracy: 0.6384 Validation_Loss: 1.0122
Epoch: 5 Total_Time: 3.5257 Average_Time_per_batch: 0.0090 Train_Accuracy: 0.7014 Train_Loss: 0.8315 Validation_Accuracy: 0.6580 Validation_Loss: 0.9627
Epoch: 6 Total_Time: 3.4675 Average_Time_per_batch: 0.0089 Train_Accuracy: 0.7338 Train_Loss: 0.7386 Validation_Accuracy: 0.6663 Validation_Loss: 0.9350
Epoch: 7 Total_Time: 3.4955 Average_Time_per_batch: 0.0089 Train_Accuracy: 0.7645